In [1]:
import requests
from bs4 import BeautifulSoup
import re
import threading
from datetime import datetime
import pandas as pd
import time
from datetime import datetime, timedelta
from concurrent.futures import ThreadPoolExecutor
import concurrent.futures

In [2]:
data = {
    "keywords": "anies",
    "since_time": "2023-09-01",
    "until_time": "2023-11-10"
}

In [3]:
url = f"https://indeks.kompas.com/?site=news&date=2023-09-01&page=100"  
response = requests.get(url)

soup = BeautifulSoup(response.text, "html.parser")
articles = soup.find_all('h3', {"class": "article__title article__title--medium"})
print(articles)

[]


In [4]:
def scrape_links(date,page_number):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36'
    }
    url = f"https://indeks.kompas.com/?site=news&date={date}&page={page_number}"
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    articles = soup.find_all('h3', {"class": "article__title article__title--medium"})
    links = []
    for article in articles:
        link = article.find('a')['href']
        links.append(link)
    print(f"Scraped {len(links)} links from page {page_number}")
    
    return links

In [5]:
def scrape_link_per_day(date, max_threads=5):
    page_number = 1
    page_links = []

    with concurrent.futures.ThreadPoolExecutor(max_threads) as executor:
        futures = []

        while True:
            future = executor.submit(scrape_links, date, page_number)
            futures.append(future)
            page_number += 1

            # Break the loop if no more articles are found
            if not future.result():
                break

        for future in concurrent.futures.as_completed(futures):
            page_links.extend(future.result())

    return page_links

In [6]:
link=scrape_link_per_day(data["since_time"])

Scraped 15 links from page 1
Scraped 15 links from page 2
Scraped 15 links from page 3
Scraped 15 links from page 4
Scraped 15 links from page 5
Scraped 15 links from page 6
Scraped 15 links from page 7
Scraped 15 links from page 8
Scraped 15 links from page 9
Scraped 15 links from page 10
Scraped 15 links from page 11
Scraped 15 links from page 12
Scraped 15 links from page 13
Scraped 15 links from page 14
Scraped 12 links from page 15
Scraped 0 links from page 16


In [7]:
print(link)

['https://nasional.kompas.com/read/2023/09/01/20590331/manuver-nasdem-duetkan-anies-cak-imin-sby-demokrat-diselamatkan-tuhan', 'https://nasional.kompas.com/read/2023/09/01/20524751/merasa-dikhianati-sby-akui-demokrat-keliru-dukung-anies', 'https://nasional.kompas.com/read/2023/09/01/20521511/tak-wajib-skripsi-kemendikbudristek-minta-mahasiswa-jangan-menggampangkan', 'https://nasional.kompas.com/read/2023/09/01/20473061/nasdem-pkb-bakal-koalisi-gerindra-singgung-soal-cak-imin-tak-keberatan', 'https://nasional.kompas.com/read/2023/09/01/20465871/sby-soal-manuver-nasdem-dan-anies-melebihi-batas-kepatutan-kasar', 'https://megapolitan.kompas.com/read/2023/09/01/20240391/warga-lenteng-agung-usulkan-contra-flow-di-titik-motor-lawan-arus', 'https://nasional.kompas.com/read/2023/09/01/20240191/gerindra-nyatakan-kerja-sama-dengan-pkb-berakhir-kkir-bubar', 'https://nasional.kompas.com/read/2023/09/01/20212101/manuver-nasdem-dan-cak-imin-disebut-potensial-bikin-gempa-politik', 'https://nasional.ko

In [8]:
def scrape_url(url,max_retries=2):
    retries = 0
    while retries < max_retries:
            try:
                headers = {
                    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36'
                }
                response = requests.get(url, headers=headers)
                if response.status_code == 200:
                    soup = BeautifulSoup(response.text, 'html.parser')
                    
                    
                    # Judul Berita
                    title_elem = soup.find('h1', {"class": "read__title"})
                    if title_elem:
                        title_text = title_elem.text.strip()
                    else:
                        title_text = "Title not found"  
                    # tanggal berita
                    date_elem = soup.find('div', {"class": "read__time"})
                    if date_elem:
                        date_text = date_elem.text.strip()
                    else:
                        date_text = "Date not found"
                    #     # Content Berita
                    body_elem = soup.find('div', {"class": "read__content"})
                        
                    if body_elem:
                        content_elem = body_elem.find_all('p')
                        content_text = ""
                        for p in content_elem:
                                content_text += p.text.strip() + "\n"
                            
                        if content_text.strip():
                            content_text = content_text.replace('\n', '').replace('\r', '').replace('\t', '')
                            content_text = ' '.join(content_text.split())
                        else:
                            content_text="Content not found"
                    else:
                            content_text="Content not found"

                    return{
                        'title': title_text,
                        'date': date_text,
                        'content':content_text,
                        'link' : url}
                elif response.status_code == 429:
                    print(f"Received a 429 error for {url}. Retrying in 5 seconds...")
                    time.sleep(5)
                else:
                    print(f"Failed to retrieve data from {url}: Status Code {response.status_code}")
            except requests.exceptions.RequestException as e:
                print(f"Error fetching URL '{url}': {e}")
            except Exception as e:
                print(f"Error processing URL '{url}': {e}")
            retries += 1
            if retries < max_retries:
                print(f"Retrying {url} (Attempt {retries}/{max_retries})")
                time.sleep(5)  # You can adjust the delay as needed
    return None         

In [9]:
def is_keyword_in_text(keyword, text):
    # Case-insensitive search for whole word match
    pattern = re.compile(r'\b{}\b'.format(re.escape(keyword)), flags=re.IGNORECASE)

    # Check for a match with or without parentheses and hyphens
    return any(pattern.search(match) for match in [text, text.replace('(', '').replace(')', ''), text.replace('-', ' ')])

for url in link:
    data_tribunnews = scrape_url(url)
    print(data["keywords"])
    
    # Assuming data_tribunnews is a dictionary with 'title', 'content', and 'link' keys
    title_contains_keyword = any(is_keyword_in_text(keyword, data_tribunnews['title']) for keyword in data["keywords"])
    content_contains_keyword = any(is_keyword_in_text(keyword, data_tribunnews['content']) for keyword in data["keywords"])

    if title_contains_keyword or content_contains_keyword:
        print("Data contains keywords:", data_tribunnews)
    else:
        print("News does not contain the specified keywords and will not be inserted into the database. URL:", data_tribunnews['link'])

anies
Data contains keywords: {'title': 'Manuver Nasdem Duetkan Anies-Cak Imin, SBY: Demokrat Diselamatkan Tuhan', 'date': 'Kompas.com - 01/09/2023, 20:59 WIB', 'content': 'BOGOR, KOMPAS.com - Ketua Majelis Tinggi Partai Demokrat Susilo Bambang Yudhoyono (SBY) merasa partainya "diselamatkan" Tuhan, terkait manuver sepihak Partai Nasdem dan bakal capres Anies Baswedan yang menggandeng Ketua Umum Partai Kebangkitan Bangsa (PKB) Muhaimin Iskandar (Cak Imin) menjadi bakal cawapres.Hal itu disampaikan SBY dalam Sidang Majelis Tinggi Partai Demokrat yang digelar di kediamannya di Puri Cikeas, Bogor, Jawa Barat, Jumat (1/9/2023)."Di sisi lain, meskipun kita dibeginikan oleh capres Anies dan mitra koalisi kita, tapi sesungguhnya kita harus bersyukur kepada Allah S.W.T., bersyukur kepada Tuhan Yang Maha Kuasa," kata SBY.SBY mengakui Partai Demokrat ditinggalkan oleh Nasdem dan Anies yang membuat kesepakatan diam-diam dengan PKB. Akan tetapi, dia masih bersyukur karena saat manuver itu terungkap

KeyboardInterrupt: 